In [3]:
import requests
import pandas as pd
import urllib.parse
import requests
import geopandas as gpd
import pandas as pd
import folium
import urllib


base_url = "https://reference.geoconnex.us"

In [13]:
url = "https://reference.geoconnex.us/collections?f=json"

# Fetch the JSON data from the URL
response = requests.get(url)
data = response.json()

# Convert the collections data to a DataFrame
collections_df = pd.DataFrame(data['collections'])
print(collections_df.head())

     id title                                        description  \
0  hu02  HU02  Two-digit Hydrologic Regions from USGS NHDPlus...   
1  hu04  HU04  Four-digit Hydrologic Subregion from USGS NHDP...   
2  hu06  HU06  Six-digit Hydrologic Basins from USGS NHDPlus ...   
3  hu08  HU08  Eight-digit Hydrologic Subbasins from USGS NHD...   
4  hu10  HU10  Ten-digit Watersheds from USGS NHDPlus High Re...   

                        keywords  \
0     [Hydrologic Regions, USGS]   
1  [Hydrologic Subregions, USGS]   
2      [Hydrologic Basins, USGS]   
3   [Hydrologic Subbasins, USGS]   
4             [Watersheds, USGS]   

                                               links  \
0  [{'type': 'application/html', 'rel': 'canonica...   
1  [{'type': 'application/html', 'rel': 'canonica...   
2  [{'type': 'application/html', 'rel': 'canonica...   
3  [{'type': 'application/html', 'rel': 'canonica...   
4  [{'type': 'application/html', 'rel': 'canonica...   

                                     

In [6]:

# Construct a query for a river name that includes the string "animas"
query = urllib.parse.quote("name_at_outlet ILIKE '%animas%'")
url = f"{base_url}/collections/mainstems/items?f=json&filter={query}"
print(url)
# Read the data into a GeoDataFrame
animas_rivers = gpd.read_file(url)

# Display the results
selected_columns = animas_rivers[['uri', 'name_at_outlet', 'outlet_drainagearea_sqkm', 'geometry']]
print(selected_columns)



https://reference.geoconnex.us/collections/mainstems/items?f=json&filter=name_at_outlet%20ILIKE%20%27%25animas%25%27
                                          uri    name_at_outlet  \
0  https://geoconnex.us/ref/mainstems/1609859  Las Animas Creek   
1  https://geoconnex.us/ref/mainstems/1888173      Animas Creek   
2    https://geoconnex.us/ref/mainstems/35394      Animas River   

   outlet_drainagearea_sqkm                                           geometry  
0                     568.1  LINESTRING (-107.83774 33.06353, -107.8366 33....  
1                    1188.8  LINESTRING (-108.77441 31.49576, -108.77676 31...  
2                    3488.9  LINESTRING (-107.613 37.91702, -107.61092 37.9...  


In [4]:
import folium
import requests
from itertools import cycle

# URL to the GeoJSON data
query = urllib.parse.quote("name_at_outlet ILIKE '%animas%'")
url = f"https://reference.geoconnex.us/collections/mainstems/items?f=json&filter={query}"

# Fetch the GeoJSON data
response = requests.get(url)
data = response.json()

# Calculate map center by averaging coordinates (assumes geometry type is LineString)
coordinates = [feature['geometry']['coordinates'] for feature in data['features']]
all_coords = [coord for sublist in coordinates for coord in sublist]  # Flatten the list
avg_lat = sum(coord[1] for coord in all_coords) / len(all_coords)
avg_lon = sum(coord[0] for coord in all_coords) / len(all_coords)
map_center = (avg_lat, avg_lon)

# Create the map
m = folium.Map(location=map_center, zoom_start=6)

# Define a list of hardcoded colors
hardcoded_colors = ['blue', 'green', 'red', 'orange', 'black', 'purple']
color_cycle = cycle(hardcoded_colors)  # Cycle through the colors

# Add GeoJSON data to the map with tooltips
for feature in data['features']:
    river_name = feature['properties']['name_at_outlet']
    color = next(color_cycle)  # Get the next color from the cycle

    folium.GeoJson(
        feature,
        name=river_name,
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': color,
            'weight': 2,
            'fillOpacity': 0.6,
        },
        tooltip=folium.GeoJsonTooltip(fields=['name_at_outlet'], aliases=['River Name:'], localize=True)
    ).add_to(m)

# Define the legend HTML
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 150px; height: auto; 
            padding: 10px; font-size: 14px; z-index:9999; 
            background-color: white; border: 2px solid grey; 
            border-radius: 5px;">
    <h4 style="margin: 0;">Legend</h4>
'''

# Add river names and colors to the legend
for feature in data['features']:
    river_name = feature['properties']['name_at_outlet']
    color = next(color_cycle)  # Get the next color for legend

    legend_html += f'''
    <div style="display: flex; align-items: center;">
        <div style="width: 20px; height: 20px; background-color: {color}; margin-right: 5px;"></div>
        <span>{river_name}</span>
    </div>
    '''

legend_html += '</div>'

# Add the legend to the map
m.get_root().html.add_child(folium.Element(legend_html))

m.save('../../../static/html/python/rivers-with-animas-in-name.html')

# Display the map
m

In [36]:
import folium
import requests
from urllib.parse import quote

# Base URL for the API
base_url = "https://reference.geoconnex.us"

# Construct a query for rivers with outlet_drainagearea_sqkm > 500,000
query = "outlet_drainagearea_sqkm > 500000"
encoded_query = quote(query)  # URL encode the query
url = f"{base_url}/collections/mainstems/items?f=json&filter={encoded_query}"

# Fetch the GeoJSON data
response = requests.get(url)
data = response.json()

# Calculate map center by averaging coordinates (assuming geometry type is LineString)
coordinates = [feature['geometry']['coordinates'] for feature in data['features']]
all_coords = [coord for sublist in coordinates for coord in sublist]  # Flatten the list
avg_lat = sum(coord[1] for coord in all_coords) / len(all_coords)
avg_lon = sum(coord[0] for coord in all_coords) / len(all_coords)
map_center = (avg_lat, avg_lon)

# Create the map
m = folium.Map(location=map_center, zoom_start=6)

# Hard-coded list of colors
colors = [
    "#FF5733",  # Red
    "#33FF57",  # Green
    "#3357FF",  # Blue
    "#FF33FF",  # Magenta
    "#FFA500",  # Orange
    "#800080",  # Purple
    "#FFC0CB",  # Pink
    "#A52A2A"   # Brown
]

# Create a dictionary to store river names and their corresponding colors
river_colors = {}

# Add GeoJSON data to the map with tooltips and individual colors
for i, feature in enumerate(data['features']):
    river_name = feature['properties']['name_at_outlet']
    
    # Assign a color based on the index
    river_colors[river_name] = colors[i % len(colors)]
    
    folium.GeoJson(
        feature,
        style_function=lambda x, color=river_colors[river_name]: {
            'color': color,
            'weight': 3,
            'opacity': 0.7,
        },
        tooltip=folium.GeoJsonTooltip(fields=['name_at_outlet', 'outlet_drainagearea_sqkm'],
                                      aliases=['River Name:', 'Drainage Area (sq km):'], localize=True)
    ).add_to(m)

# Create the legend with separate names for each river
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 150px; height: auto; 
            padding: 10px; font-size: 14px; z-index:9999; 
            background-color: white; border: 2px solid grey; 
            border-radius: 5px;">
    <h4 style="margin: 0;">Legend</h4>
'''

# Iterate through river colors to create legend items
for river_name, color in river_colors.items():
    legend_html += f'''
    <div style="display: flex; align-items: center;">
        <div style="width: 20px; height: 20px; background-color: {color}; margin-right: 5px;"></div>
        <span>{river_name}</span>
    </div>
    '''

legend_html += '''
</div>
'''

# Add the legend to the map
m.get_root().html.add_child(folium.Element(legend_html))

# Save the map
m.save('../../../static/html/python/large-rivers.html')

# Display the map
m


In [16]:
import requests
import pandas as pd

# Animas River mainstem URI
animas_uri = "https://geoconnex.us/ref/mainstems/35394"

# Fetch the Animas River data
response = requests.get(animas_uri, params={'f': 'json'})
animas_data = response.json()

# Extract datasets
datasets = animas_data['properties']['datasets']

# Create a DataFrame
df_datasets = pd.DataFrame(datasets)

# Display the DataFrame with options similar to datatable in R
print(df_datasets.head(5))  # Display the first 5 rows


                                  monitoringLocation       siteName  \
0  https://sta.geoconnex.dev/collections/USGS/Thi...  USGS-09363500   
1  https://sta.geoconnex.dev/collections/USGS/Thi...  USGS-09363500   
2  https://sta.geoconnex.dev/collections/USGS/Thi...  USGS-09363500   
3  https://sta.geoconnex.dev/collections/USGS/Thi...  USGS-09363500   
4  https://sta.geoconnex.dev/collections/USGS/Thi...  USGS-09363500   

                                  datasetDescription    type  \
0  [Turbidity, FNU / USGS-09363500-1f7260408f1d4c...  Stream   
1  [Turbidity, FNU / USGS-09363500-1f7260408f1d4c...  Stream   
2  Specific cond at 25C / USGS-09363500-252c7ef5e...  Stream   
3  Specific cond at 25C / USGS-09363500-252c7ef5e...  Stream   
4  None / USGS-09363500-3177e767c0af430b9efae7195...  Stream   

                                                 url  \
0  https://waterdata.usgs.gov/monitoring-location...   
1  https://waterdata.usgs.gov/monitoring-location...   
2  https://waterdata

In [37]:
# SPARQL endpoint
endpoint = "https://graph.geoconnex.us"

# Revised SPARQL query
query = '''
PREFIX schema: <https://schema.org/>
PREFIX gsp: <http://www.opengis.net/ont/geosparql#>
PREFIX hyf: <https://www.opengis.net/def/schema/hy_features/hyf/>
SELECT DISTINCT ?monitoringLocation ?siteName ?datasetDescription ?type ?url
       ?variableMeasured ?variableUnit ?measurementTechnique ?temporalCoverage
       ?distributionName ?distributionURL ?distributionFormat ?wkt
WHERE {
  VALUES ?mainstem { <https://geoconnex.us/ref/mainstems/35394> }

  ?monitoringLocation hyf:referencedPosition/hyf:HY_IndirectPosition/hyf:linearElement ?mainstem ;
                      schema:subjectOf ?item ;
                      hyf:HydroLocationType ?type ;
                      gsp:hasGeometry/gsp:asWKT ?wkt .

  ?item schema:name ?siteName ;
        schema:temporalCoverage ?temporalCoverage ;
        schema:url ?url ;
        schema:variableMeasured ?variableMeasured .

  ?variableMeasured schema:description ?datasetDescription ;
                    schema:name ?variableMeasuredName ;
                    schema:unitText ?variableUnit ;
                    schema:measurementTechnique ?measurementTechnique .

  OPTIONAL {
    ?item schema:distribution ?distribution .
    ?distribution schema:name ?distributionName ;
                  schema:contentUrl ?distributionURL ;
                  schema:encodingFormat ?distributionFormat .
  }

  FILTER(REGEX(?datasetDescription, "temperature", "i"))
}
ORDER BY ?siteName
'''

# Execute the SPARQL query
response = requests.get(
    endpoint,
    params={'query': query},
    headers={'Accept': 'application/sparql-results+json'}
)

# Parse the JSON response
result = response.json()

# Extract the results into a DataFrame
datasets = pd.json_normalize(result['results']['bindings'])

print(datasets.head(5))

  monitoringLocation.type                           monitoringLocation.value  \
0                     uri  https://sta.geoconnex.dev/collections/USGS/Thi...   
1                     uri  https://sta.geoconnex.dev/collections/USGS/Thi...   
2                     uri  https://sta.geoconnex.dev/collections/USGS/Thi...   
3                     uri  https://sta.geoconnex.dev/collections/USGS/Thi...   
4                     uri  https://sta.geoconnex.dev/collections/USGS/Thi...   

  siteName.type siteName.value datasetDescription.type  \
0       literal  USGS-09359020                 literal   
1       literal  USGS-09359020                 literal   
2       literal  USGS-09363500                 literal   
3       literal  USGS-09363500                 literal   
4       literal  USGS-09364500                 literal   

  datasetDescription.value type.type type.value url.type  \
0      Temperature in degC   literal     Stream  literal   
1      Temperature in degC   literal     Stream  l

In [11]:
# Animas River mainstem URI
animas_uri = "https://geoconnex.us/ref/mainstems/35394"
base_url = "https://reference.geoconnex.us"  

# Function to query features by mainstem URI
def query_by_mainstem(collection, mainstem_uri):
    # Correctly format the query without encoding the URL
    query = f"mainstem_uri = '{mainstem_uri}'"
    url = f"{base_url}/collections/{collection}/items?f=json&filter={query}"
    
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad responses
    return pd.json_normalize(response.json()['features'])  # Flatten the JSON response

# Query dams and gages
animas_dams = query_by_mainstem("dams", animas_uri)
animas_gages = query_by_mainstem("gages", animas_uri)

print(animas_dams)

      type       id                         properties.uri properties.name  \
0  Feature  1077435  https://geoconnex.us/ref/dams/1077435            None   
1  Feature  1077436  https://geoconnex.us/ref/dams/1077436            None   

  properties.subjectof properties.provider_id  \
0                 None                NM00520   
1                 None                NM00655   

                     properties.nhdpv2_reachcode_uri  \
0  https://geoconnex.us/nhdplusv2/reachcode/14080...   
1  https://geoconnex.us/nhdplusv2/reachcode/14080...   

   properties.nhdpv2_reach_measure  properties.drainage_area_sqkm_nhdpv2  \
0                          39.9115                             3258.5049   
1                          42.5225                             3258.5049   

                    properties.mainstem_uri  \
0  https://geoconnex.us/ref/mainstems/35394   
1  https://geoconnex.us/ref/mainstems/35394   

                              properties.description  properties.fid  \
0  Re

In [41]:
import pandas as pd
import requests
import folium

# Step 1: Find all dams named "Hoover"
query_hoover = "name LIKE '%Hoover%'"
url_hoover = f"{base_url}/collections/dams/items?f=json&filter={query_hoover}"
response_hoover = requests.get(url_hoover)
response_hoover.raise_for_status()  # Raise an error for bad responses
hoover_dams = pd.json_normalize(response_hoover.json()['features'])  # Flatten the JSON response

# Print the number of Hoover dams
print("Number of dams named 'Hoover':", len(hoover_dams))

# Create an interactive table of all Hoover dams
hoover_dams_table = hoover_dams[['properties.uri', 'properties.name', 'properties.drainage_area_sqkm']]
hoover_dams_table.columns = ['URI', 'Name', 'Drainage Area (sq km)']
hoover_dams_table = hoover_dams_table.sort_values(by='Drainage Area (sq km)', ascending=False)

# Step 2: Query for large Hoover dams using a combined filter
query_large_hoover = "name LIKE '%Hoover%' AND drainage_area_sqkm > 100000"
url_large_hoover = f"{base_url}/collections/dams/items?f=json&filter={query_large_hoover}"
response_large_hoover = requests.get(url_large_hoover)
response_large_hoover.raise_for_status()  # Raise an error for bad responses
large_hoover_dams = pd.json_normalize(response_large_hoover.json()['features'])  # Flatten the JSON response

# Print the number of large Hoover dams
print("\nNumber of large Hoover dams (Drainage Area > 100,000 sq km):", len(large_hoover_dams))

# Create an interactive table of large Hoover dams
large_hoover_dams_table = large_hoover_dams[['properties.uri', 'properties.name', 'properties.drainage_area_sqkm']]
large_hoover_dams_table.columns = ['URI', 'Name', 'Drainage Area (sq km)']
large_hoover_dams_table = large_hoover_dams_table.sort_values(by='Drainage Area (sq km)', ascending=False)

# Step 3: Create a map view of all Hoover dams
m = folium.Map(location=[37.0, -96.0], zoom_start=4)  # Adjust the initial map view as needed

# Add all Hoover dams to the map
for _, dam in hoover_dams.iterrows():
    folium.Marker(
        location=[dam['geometry.coordinates'][1], dam['geometry.coordinates'][0]],
        popup=f"{dam['properties.name']}: {dam['properties.uri']}",
    ).add_to(m)

# Highlight large Hoover dams in red
for _, large_dam in large_hoover_dams.iterrows():
    folium.Marker(
        location=[large_dam['geometry.coordinates'][1], large_dam['geometry.coordinates'][0]],
        popup=f"{large_dam['properties.name']}: {large_dam['properties.uri']}",
        icon=folium.Icon(color='red')
    ).add_to(m)

# Save the map to an HTML file
m.save("../../../static/html/python/hoover-dams.html")

m

Number of dams named 'Hoover': 39

Number of large Hoover dams (Drainage Area > 100,000 sq km): 1


In [58]:
import geopandas as gpd
import folium
from shapely.geometry import box

# Define the base URL
base_url = "https://reference.geoconnex.us"

# Define the bounding box for the Four Corners area
# Format: (min Longitude, min Latitude, max Longitude, max Latitude)
bbox = (-109.5, 36.5, -107.5, 37.5)

# Construct the URL with the bbox parameter
url = f"{base_url}/collections/pws/items?f=json&bbox={','.join(map(str, bbox))}"

# Read the data into a GeoDataFrame
four_corners_pws = gpd.read_file(url)

# Display summary of the results
print("Number of Public Water Systems found:", len(four_corners_pws))

# Create a Folium map centered around the bounding box
m = folium.Map(location=[(bbox[1] + bbox[3]) / 2, (bbox[0] + bbox[2]) / 2], zoom_start=8)

# Define a color scale based on population served
def get_color(population):
    if population < 1000:
        return 'green'
    elif population < 5000:
        return 'yellow'
    elif population < 10000:
        return 'orange'
    else:
        return 'red'

# Add GeoJson layers for each public water system
for _, row in four_corners_pws.iterrows():
    folium.GeoJson(
        row.geometry,
        tooltip=f"{row['pws_name']}<br>Population Served: {row['population_served_count']}",
        style_function=lambda feature, population=row['population_served_count']: {
            'fillColor': get_color(population),
            'color': 'black',
            'weight': 2,
            'fillOpacity': 0.6,
        }
    ).add_to(m)

# Draw the bounding box on the map
bounding_box = box(bbox[0], bbox[1], bbox[2], bbox[3])
folium.GeoJson(bounding_box).add_to(m)

# Create a legend
legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; 
                width: 150px; height: auto; 
                border:2px solid grey; 
                z-index:9999; 
                font-size:14px;
                background-color:white;
                padding: 10px;">
    <b>Population Served</b><br>
    <i style="background: green;">&nbsp;&nbsp;&nbsp;</i> Less than 1,000<br>
    <i style="background: yellow;">&nbsp;&nbsp;&nbsp;</i> 1,000 - 5,000<br>
    <i style="background: orange;">&nbsp;&nbsp;&nbsp;</i> 5,000 - 10,000<br>
    <i style="background: red;">&nbsp;&nbsp;&nbsp;</i> More than 10,000<br>
    </div>
'''

# Add the legend to the map
m.get_root().html.add_child(folium.Element(legend_html))

# Display the map
m.save("../../../static/html/python/four-corners-pws.html")
m  # This will display the map in a Jupyter Notebook, if you are using one.


Number of Public Water Systems found: 75


In [71]:
import requests
import geopandas as gpd
import json
import folium

# Define the process endpoint
process_url = "https://reference.geoconnex.us/processes/intersector/execution"

# Define the input parameters
input_params = {
    "inputs": {
        "url": "https://geoconnex.us/ref/mainstems/35394",
        "collection": "counties"
    }
}

# Execute the process
response = requests.post(
    url=process_url,
    data=json.dumps(input_params),
    headers={"Content-Type": "application/json"}
)

# Check if the request was successful
if response.status_code == 200:
    # Convert the result to a GeoDataFrame
    intersecting_counties = gpd.read_file(response.text)

    # Drop geometry and select relevant columns
    intersecting_counties_table = intersecting_counties[['name', 'uri']]
    
    # Create a DataFrame display (in a Jupyter Notebook, use display() for better formatting)
    print(intersecting_counties_table)

    # Fetch the Animas River geometry
    animas_river = gpd.read_file("https://geoconnex.us/ref/mainstems/35394")

    # Create a map view of the results
    m = folium.Map(location=[intersecting_counties.geometry.centroid.y.mean(), 
                              intersecting_counties.geometry.centroid.x.mean()], 
                    zoom_start=8)

    # Add intersecting counties to the map
    folium.GeoJson(intersecting_counties).add_to(m)

    # Add Animas River to the map
    folium.GeoJson(animas_river, style_function=lambda x: {'color': 'blue'}).add_to(m)

    # Create a custom legend
    legend_html = """
    <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 200px; height: auto; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color: white;
                 padding: 10px;">
    <h4>Intersecting Counties</h4>
    """
    
    for _, row in intersecting_counties.iterrows():
        legend_html += f'<div style="color: black;">{row["name"]}</div>'
    
    legend_html += "</div>"

    # Add the legend to the map
    m.get_root().html.add_child(folium.Element(legend_html))

    # Save map to an HTML file
    m.save('../../../static/html/python/intersect-url.html')
else:
    print(f"Error: {response.status_code}, {response.text}")

       name                                      uri
0  La Plata  https://geoconnex.us/ref/counties/08067
1  San Juan  https://geoconnex.us/ref/counties/08111
2  San Juan  https://geoconnex.us/ref/counties/35045


/var/folders/ff/zdlbl2jn4jx2h5qfdzjrdqvc0000gn/T/ipykernel_20050/1187489404.py:39: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[intersecting_counties.geometry.centroid.y.mean(),
/var/folders/ff/zdlbl2jn4jx2h5qfdzjrdqvc0000gn/T/ipykernel_20050/1187489404.py:40: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  intersecting_counties.geometry.centroid.x.mean()],


In [1]:
import requests
import geopandas as gpd
import json
import folium

# Define the process endpoint
process_url = "https://reference.geoconnex.us/processes/intersector/execution"

# Define the input parameters
input_params = {
    "inputs": {
        "url": "https://geoconnex.us/ref/mainstems/35394",
        "collection": "counties"
    }
}

# Execute the process
response = requests.post(
    url=process_url,
    data=json.dumps(input_params),
    headers={"Content-Type": "application/json"}
)

# Convert the result to a GeoDataFrame
intersecting_counties = gpd.read_file(response.text)

# Drop geometry and select relevant columns
intersecting_counties_table = intersecting_counties[['name', 'uri']]

# Create a DataFrame display (in a Jupyter Notebook, use display() for better formatting)
print(intersecting_counties_table)

# Fetch the Animas River geometry
animas_river = gpd.read_file("https://geoconnex.us/ref/mainstems/35394")

# Create a map view of the results
m = folium.Map(location=[intersecting_counties.geometry.centroid.y.mean(), 
                            intersecting_counties.geometry.centroid.x.mean()], 
                zoom_start=8)

# Add intersecting counties to the map
folium.GeoJson(intersecting_counties).add_to(m)

# Add Animas River to the map
folium.GeoJson(animas_river, style_function=lambda x: {'color': 'blue'}).add_to(m)

# Create a custom legend
legend_html = """
<div style="position: fixed; 
                bottom: 50px; left: 50px; width: 200px; height: auto; 
                border:2px solid grey; z-index:9999; font-size:14px;
                background-color: white;
                padding: 10px;">
<h4>Intersecting Counties</h4>
"""

for _, row in intersecting_counties.iterrows():
    legend_html += f'<div style="color: black;">{row["name"]}</div>'

legend_html += "</div>"

# Add the legend to the map
m.get_root().html.add_child(folium.Element(legend_html))
m.save('../../../static/html/python/intersect-url.html')
m


       name                                      uri
0  La Plata  https://geoconnex.us/ref/counties/08067
1  San Juan  https://geoconnex.us/ref/counties/08111
2  San Juan  https://geoconnex.us/ref/counties/35045


/var/folders/ff/zdlbl2jn4jx2h5qfdzjrdqvc0000gn/T/ipykernel_45749/778465719.py:37: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[intersecting_counties.geometry.centroid.y.mean(),
/var/folders/ff/zdlbl2jn4jx2h5qfdzjrdqvc0000gn/T/ipykernel_45749/778465719.py:38: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  intersecting_counties.geometry.centroid.x.mean()],


In [5]:
import requests
import pandas as pd
import geopandas as gpd
from urllib.parse import quote
import folium
from folium import Element

# Base URL
base_url = "https://reference.geoconnex.us"

# Animas River mainstem URI
animas_uri = "https://geoconnex.us/ref/mainstems/35394"

# Function to query features by mainstem URI
def query_by_mainstem(collection, mainstem_uri):
    query = f"mainstem_uri = '{mainstem_uri}'"
    url = f"{base_url}/collections/{collection}/items?f=json&filter={quote(query)}"
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad responses
    return gpd.read_file(url)

# Query dams and gages
animas_dams = query_by_mainstem("dams", animas_uri)
animas_gages = query_by_mainstem("gages", animas_uri)

# Function to create a hyperlink in HTML
def create_link(url):
    if pd.notna(url) and url != "":
        return f'<a href="{url}" target="_blank">Link</a>'
    return "N/A"

# Create interactive tables
animas_dams['subjectof'] = animas_dams['subjectof'].apply(create_link)

# Use HTML for display
animas_dams_html = animas_dams[['uri', 'name', 'subjectof']].to_html(escape=False, index=False)
animas_gages['subjectof'] = animas_gages['subjectof'].apply(create_link)
animas_gages_html = animas_gages[['uri', 'name', 'subjectof']].to_html(escape=False, index=False)

# # Display data tables (this would typically be in a Jupyter Notebook or web app)
# print("Dams on the Animas River:")
# display(HTML(animas_dams_html))  # Use display from IPython.display if in Jupyter
# print("Gages on the Animas River:")
# display(HTML(animas_gages_html))  # Use display from IPython.display if in Jupyter

# Fetch the Animas River geometry
animas_river = gpd.read_file(animas_uri)

# Create a map view of the results
m = folium.Map(location=[animas_river.geometry.centroid.y, animas_river.geometry.centroid.x], zoom_start=10)

# Add Animas River
folium.GeoJson(animas_river, style_function=lambda x: {'color': 'blue', 'weight': 2}).add_to(m)

# Add dams
for _, row in animas_dams.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['subjectof'],
        icon=folium.Icon(color='red', icon='water')
    ).add_to(m)

# Add gages
for _, row in animas_gages.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['subjectof'],
        icon=folium.Icon(color='green', icon='info-sign')
    ).add_to(m)

# Create a legend
legend_html = """
<div style="position: fixed; 
     top: 10px; right: 10px; width: 150px; height: auto; 
     border:2px solid grey; background-color: white; 
     z-index: 1000; padding: 10px;">
     <h4>Legend</h4>
     <i class="fa fa-circle" style="color:red"></i>&nbsp; Dams<br>
     <i class="fa fa-circle" style="color:green"></i>&nbsp; Gages<br>
</div>
"""

# Add legend to map
m.get_root().html.add_child(Element(legend_html))
m.save('../../../static/html/python/gages-on-river.html')
m

/var/folders/ff/zdlbl2jn4jx2h5qfdzjrdqvc0000gn/T/ipykernel_45749/3852601569.py:50: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[animas_river.geometry.centroid.y, animas_river.geometry.centroid.x], zoom_start=10)
/Users/cloftus/github/docs.geoconnex.us/.venv/lib/python3.12/site-packages/folium/utilities.py:94: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(coord)
/Users/cloftus/github/docs.geoconnex.us/.venv/lib/python3.12/site-packages/folium/utilities.py:100: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if math.isnan(float(coord)):
/Users/cloftus/github/docs.geoconnex.us/.venv/lib/python3.12/site-packages/folium/utilitie

In [6]:
import requests
import json
import pandas as pd

# SPARQL endpoint
endpoint = "https://graph.geoconnex.us"

# Revised SPARQL query
query = """
PREFIX schema: <https://schema.org/>
PREFIX gsp: <http://www.opengis.net/ont/geosparql#>
PREFIX hyf: <https://www.opengis.net/def/schema/hy_features/hyf/>
SELECT DISTINCT ?monitoringLocation ?siteName ?datasetDescription ?type ?url
       ?variableMeasured ?variableUnit ?measurementTechnique ?temporalCoverage
       ?distributionName ?distributionURL ?distributionFormat ?wkt
WHERE {
  VALUES ?mainstem { <https://geoconnex.us/ref/mainstems/35394> }

  ?monitoringLocation hyf:referencedPosition/hyf:HY_IndirectPosition/hyf:linearElement ?mainstem ;
                      schema:subjectOf ?item ;
                      hyf:HydroLocationType ?type ;
                      gsp:hasGeometry/gsp:asWKT ?wkt .

  ?item schema:name ?siteName ;
        schema:temporalCoverage ?temporalCoverage ;
        schema:url ?url ;
        schema:variableMeasured ?variableMeasured .

  ?variableMeasured schema:description ?datasetDescription ;
                    schema:name ?variableMeasuredName ;
                    schema:unitText ?variableUnit ;
                    schema:measurementTechnique ?measurementTechnique .

  OPTIONAL {
    ?item schema:distribution ?distribution .
    ?distribution schema:name ?distributionName ;
                  schema:contentUrl ?distributionURL ;
                  schema:encodingFormat ?distributionFormat .
  }

  # Filter datasets by the desired variable description
  FILTER(REGEX(?datasetDescription, "temperature", "i"))
}
ORDER BY ?siteName
"""

# Execute the SPARQL query
response = requests.get(
    endpoint,
    params={'query': query},
    headers={'Accept': 'application/sparql-results+json'}
)

# Parse the JSON response
result = response.json()

# Extract the results
bindings = result['results']['bindings']
datasets = pd.json_normalize(bindings)

datasets

,monitoringLocation.type,monitoringLocation.value,siteName.type,siteName.value,datasetDescription.type,datasetDescription.value,type.type,type.value,url.type,url.value,...,temporalCoverage.value,distributionName.type,distributionName.value,distributionURL.type,distributionURL.value,distributionFormat.type,distributionFormat.value,wkt.datatype,wkt.type,wkt.value
0,uri,https://sta.geoconnex.dev/collections/USGS/Thi...,literal,USGS-09359020,literal,Temperature in degC,literal,Stream,literal,https://waterdata.usgs.gov/monitoring-location...,...,2024-08-30T09:15:00Z/2024-09-09T18:00:00Z,literal,USGS SensorThings API,literal,https://labs.waterdata.usgs.gov/sta/v1.1/Datas...,literal,application/json,http://www.opengis.net/ont/geosparql#wktLiteral,literal,POINT (-107.668222222222 37.7883333333333)
1,uri,https://sta.geoconnex.dev/collections/USGS/Thi...,literal,USGS-09359020,literal,Temperature in degC,literal,Stream,literal,https://waterdata.usgs.gov/monitoring-location...,...,2024-08-30T09:15:00Z/2024-09-09T18:00:00Z,literal,USGS Instantaneous Values Service,literal,https://waterservices.usgs.gov/nwis/iv/?sites=...,literal,text/tab-separated-values,http://www.opengis.net/ont/geosparql#wktLiteral,literal,POINT (-107.668222222222 37.7883333333333)
2,uri,https://sta.geoconnex.dev/collections/USGS/Thi...,literal,USGS-09363500,literal,Temperature in degC,literal,Stream,literal,https://waterdata.usgs.gov/monitoring-location...,...,2024-08-30T09:15:00Z/2024-09-09T18:00:00Z,literal,USGS SensorThings API,literal,https://labs.waterdata.usgs.gov/sta/v1.1/Datas...,literal,application/json,http://www.opengis.net/ont/geosparql#wktLiteral,literal,POINT (-107.87533333333333 37.036569444444446)
3,uri,https://sta.geoconnex.dev/collections/USGS/Thi...,literal,USGS-09363500,literal,Temperature in degC,literal,Stream,literal,https://waterdata.usgs.gov/monitoring-location...,...,2024-08-30T09:15:00Z/2024-09-09T18:00:00Z,literal,USGS Instantaneous Values Service,literal,https://waterservices.usgs.gov/nwis/iv/?sites=...,literal,text/tab-separated-values,http://www.opengis.net/ont/geosparql#wktLiteral,literal,POINT (-107.87533333333333 37.036569444444446)
4,uri,https://sta.geoconnex.dev/collections/USGS/Thi...,literal,USGS-09364500,literal,Temperature in degC,literal,Stream,literal,https://waterdata.usgs.gov/monitoring-location...,...,2024-08-30T09:15:00Z/2024-09-09T18:30:00Z,literal,USGS SensorThings API,literal,https://labs.waterdata.usgs.gov/sta/v1.1/Datas...,literal,application/json,http://www.opengis.net/ont/geosparql#wktLiteral,literal,POINT (-108.20175 36.7225)
5,uri,https://sta.geoconnex.dev/collections/USGS/Thi...,literal,USGS-09364500,literal,Temperature in degC,literal,Stream,literal,https://waterdata.usgs.gov/monitoring-location...,...,2024-08-30T09:15:00Z/2024-09-09T18:30:00Z,literal,USGS Instantaneous Values Service,literal,https://waterservices.usgs.gov/nwis/iv/?sites=...,literal,text/tab-separated-values,http://www.opengis.net/ont/geosparql#wktLiteral,literal,POINT (-108.20175 36.7225)
